In [25]:
#-*-coding utf-8 -*-
%run -i modulo_busca_arquivos.py
import json, ast, uuid
from ipywidgets import widgets, Box, VBox, HBox, Button, Layout, Checkbox, Label
from IPython.display import HTML, Javascript, display, display_javascript, display_html
        
css = """
<style> 
.container{
        width: 98%;
    }

.output{
    height:100%;
    }

.output_scroll{
    height: 100%;
    }

.widget-label{
        height: 100px;
        white-space: wrap;
        font-weight: bold;
        font-size: 16px;
        text-align: center;
        }

.widget-checkbox {
        width: 95%;
    }
        
.space{
    text-align: left;
    font-size: 13px;
    }

.vazio{
    color: red;
    font-size: 12px;
    text-align: left;
    }

.blue{
    color: blue;
    font-size: 11.5px;
    }

.vertical-line{
    border-right: 1px groove black;
    }
    
.horizontal-line{
    border-bottom: 1px groove black;
    }
    
.top-line{
    border-top: 1px groove black;
    }
</style>"""

emExibicao = Label(value="Exibindo "+str(len(retorno.keys()))+" de "+str(len(retorno.keys()))+" arquivos", description="Quantidade de Arquivos")
unicos = list()
multiplos = dict()
filtro = dict()
cells_visable = True

selectMultiple = widgets.SelectMultiple(layout=Layout(
                        width='99.75%',
                        flex_flow='column',
                        display="flex",
                        align_items='inherit'))

def toggle_code_cells(btn):
    global cells_visable
    if cells_visable:
        display(Javascript("$('div.input').hide();"))
        btn.description = "Mostrar Celulas"
    else:
        display(Javascript("$('div.input').show();"))
        btn.description = "Esconder Celulas"
    cells_visable = not cells_visable
    
toggle_cell = Button(description="Esconder Celulas", icon="fa-eye-slash", button_style="danger", layout=Layout(width="149px", align_self="center", display="flex-shrink"))
toggle_cell.style.button_color ="#FF3030"
toggle_cell.on_click(toggle_code_cells)

def recolheMenu(n):
    if(left_box.layout.visibility == "hidden"):
        vBox.layout.width = "50%"
        left_box.layout.visibility = "visible"
        left_box.layout.width = "50%"
    else:
        left_box.layout.visibility = "hidden"
        left_box.layout.width = "0%"
        vBox.layout.width = "100%"
        
def collapseUnicas(n):
    if(static.layout.visibility == "hidden"):
        collapse.icon="fa-angle-double-up"
        collapse.tooltip="Collapse"
        static.layout.height = "100%"
        static.layout.visibility = "visible"
    else:
        static.layout.visibility = "hidden"
        static.layout.height = "0"
        collapse.tooltip="Expand"
        collapse.icon="fa-angle-double-down"
        
def renderJson(n):
    selectMultiple.options


        
def change(x):
    global filtro
    key = x.owner.style.description_width
    value = x.owner.description
    if x.new:   
        if not key in filtro.keys():
            filtro[key] = list()
        if not value in filtro[key]:
            filtro[key].append(value)    
    else:
        if len(filtro[key]) == 1:
            del(filtro[key])
        else:
            filtro[key].remove(value)

toggle_filter = Button(description="Esconder Filtros", icon="fa-eye-slash", button_style="info", layout=Layout(align_self="center", display="flex-wrap"))
toggle_filter.style.button_color = "#4D4D4D"
toggle_filter.on_click(recolheMenu)


def aplicaFiltros(n):
    global filtro
    global emExibicao    
    af = ApplyFilters(filtro, retorno)
    filtrado = af.make()    
    capturados = set()
    for k in filtrado.keys():
        for v in filtrado[k]:
            capturados.add(v)
    
    if len(filtrado.keys()) != 0:
        emExibicao.value = value="Exibindo "+str(len(capturados))+" de "+str(len(retorno.keys()))+" arquivos"
    
    montar_accordions(capturados)
        
def montar_accordions(jsons):
    applyChooses = Button(description="Aplicar", button_style="primary" ,icon="fa-check-circle", layout=Layout(align_self="flex-end", display="flex-end"))
    applyChooses.style.button_color="#009850"
    applyChooses.on_click(renderJson)
#     accordions = list()
#     tab_contents = []
    items = list()


    for i in jsons:
        for j in retorno:
            if i == retorno[j]['file']:  
#                 path = str(i).split("/")
#                 accord = widgets.HBox()
#                 label = widgets.Label(value=str(i))
#                 multKey = set(retorno[j]['config']) & set(t['multiple'])
#                 children=[label]
# #                 children=[RenderJSON(v) for v in retorno[j]['content']]
#                 accord.children = children
# #                 accord.set_title(0, str(i))
#                 accordions.append(accord)
    
            
                items.append(i)
            
    
    selectMultiple.options=[i for i in items]
    mainBox.children = [VBox([Label(value='Arquivos Retornados:'), selectMultiple, applyChooses], layout=Layout(width="99%", height="40%"))]


box_layout = Layout(width="100%", 
                    display="flex",
                    flex_direction="column",
                    border="2px groove grey",
                    align_items='stretch')

vBox_layout = Layout(width="50%",
                    align_content="flex-start",
                    display="flex")

left_layout = Layout(width="50%", flex_flow="column", display="flex", align_items="flex-start", align_self="center")
right_layout = Layout(width="99%", flex_flow="row", justify_content="space-between", display="flex", align_items="flex-end", align_self="center")
main_layout = Layout(width="90%", top="1%", align_self="center", display="justify-content")

filter_btn = Button(description="Aplicar Filtros", icon="fa-paper-plane", button_style="primary", layout=Layout(align_self="flex-end", display = "flex-wrap"))
filter_btn.style.button_color = '#009850'#'#2CD660'
filter_btn.on_click(aplicaFiltros)

uniKey = t['unique'].keys()
for i in uniKey:
    i = str(i.encode("ascii", "replace"))
    unicos.append(Label(value=str(i)+": "+str(t['unique'][i]).encode("ascii", "replace"), tooltip=str(t['unique'][i]), description=str(t['unique'][i]), display="flex-wrap", width="95%", style={'description_width':'initial'}).add_class('space').add_class('blue')) 
    
    
multKey = t['multiple'].keys()
for j in multKey:
    listaDeCheckBox = list()    
    for k in t['multiple'][j]:
        option = Checkbox(description=unicode(str(k), encoding="ascii") , width="100%", display='flex', indent=False, style={"description_width": str(j)}).add_class('space')
        option.observe(change, names='value')
        listaDeCheckBox.append(option)
    
    multiplos[j] = listaDeCheckBox  

## Criação dos Labels que serão exibidos para informar quais as propriedas que possuem filtros que podem ser aplicados. A verificação abaixo avalia se, não houver nenhuma 
## propriedade comum entre os logs, na caixa de propriedades únicas deverá ser exibida a mensagem Não Há, para indicar que todas as propriedades são múltiplas

if len(unicos) == 0:
    unicos.append(Label(value="Nao Ha Caracteristicas Unicas", description="Nao ha",  display="flex-shrink", layout=Layout(width="95%", align_self="center")).add_class('vazio'))
             

## Criação das Caixas que irão conter os valores possíveis para a aplicação dos filtros. A box static contém somente os valores que são comuns entre todos os arquivos e,logo, 
## não podem ser modificados
static = HBox([i for i in unicos],layout=Layout(width="95%", 
                               flex_flow="column",
                               display="inline-flex",
                               align_self="center"))

multiplas = list()
for k, v in multiplos.iteritems():
    multiplas.append(Label(value=str(k).capitalize(), description=str(k).capitalize(), display="flex-shrink", layout=Layout(width="95%", align_self="center")).add_class("space"))
    multiplas.append(HBox([sv for sv in v], layout=Layout(width="100%",
                                            flex_flow="column",
                                            display="inline-flex")))


## left_box é a box onde os todos os filtros serão exibidos, e será posicionada ao lado esquerdo da tela de aplicação.
multiple_box = VBox(layout=Layout(width="90%",
                                         flex_direction="column",
                                         display="flex"), style={'description_width':'initial'})
multiple_box.children = [i for i in multiplas]


collapse = Button(description="", tooltip="Collapse", icon="fa-angle-double-up", layout=Layout(width="30px", align_self="flex-end", display="flex-wrap"))
collapse.style.button_color = "white"
collapse.on_click(collapseUnicas)

uniquePlusCollapse = HBox([ Label(value="Caracteristicas Unicas", description="Caracteristicas Unicas", display="flex-shrink", layout=Layout(width="100%", align_self="flex-start")), collapse])
uniquePlusCollapse.layout=Layout(width="100%")

left_box = VBox([Label(value="Filtros", description="Filtros", display="flex-shrink", layout=Layout(width="100%", align_self="center")),
                uniquePlusCollapse,
                 static,
                 Label(value="Caracteristicas Multiplas", description="Caracteristicas Multiplas", display="flex-shrink", layout=Layout(width="91%", align_self="flex-start", left="2%")).add_class('top-line'),
                 multiple_box,
                 filter_btn]).add_class('vertical-line')

left_box.layout = left_layout

right_box = VBox(children=[toggle_filter, emExibicao, toggle_cell]).add_class('horizontal-line')
right_box.layout=right_layout

mainBox = VBox()
mainBox.layout=main_layout

vBox = VBox(children=[right_box, mainBox])
vBox.layout = vBox_layout

box = Box(children=[left_box, vBox])
box.layout = box_layout

display(HTML(css))
box

Insira o nome da pasta que deseja abrir os logs: 



Box(children=(VBox(children=(Label(value=u'Filtros', description=u'Filtros', layout=Layout(align_self=u'center', width=u'100%')), HBox(children=(Label(value=u'Caracteristicas Unicas', description=u'Caracteristicas Unicas', layout=Layout(align_self=u'flex-start', width=u'100%')), Button(icon=u'angle-double-up', layout=Layout(align_self=u'flex-end', display=u'flex-wrap', width=u'30px'), style=ButtonStyle(button_color='white'), tooltip=u'Collapse')), layout=Layout(width=u'100%')), HBox(children=(Label(value=u'hidden_activation_function: tanh', description=u'tanh', style=DescriptionStyle(description_width=u'initial'), _dom_classes=('space', 'blue')), Label(value=u'word_filters: [data.Filters.TransformLowerCaseFilter, data.Filters.TransformNumberToZeroFilter]', description=u'[data.Filters.TransformLowerCaseFilter, data.Filters.TransformNumberToZeroFilter]', style=DescriptionStyle(description_width=u'initial'), _dom_classes=('space', 'blue')), Label(value=u'start_symbol: </s>', description=u'</s>', style=DescriptionStyle(description_width=u'initial'), _dom_classes=('space', 'blue')), Label(value=u'shuffle: True', description=u'True', style=DescriptionStyle(description_width=u'initial'), _dom_classes=('space', 'blue')), Label(value=u'end_symbol: </s>', description=u'</s>', style=DescriptionStyle(description_width=u'initial'), _dom_classes=('space', 'blue')), Label(value=u'char_window_size: 5', description=u'5', style=DescriptionStyle(description_width=u'initial'), _dom_classes=('space', 'blue')), Label(value=u'word_emb_size: 100', description=u'100', style=DescriptionStyle(description_width=u'initial'), _dom_classes=('space', 'blue')), Label(value=u'decay: NORMAL', description=u'NORMAL', style=DescriptionStyle(description_width=u'initial'), _dom_classes=('space', 'blue')), Label(value=u'char_emb_size: 10', description=u'10', style=DescriptionStyle(description_width=u'initial'), _dom_classes=('space', 'blue')), Label(value=u'char_struct_grad: False', description=u'False', style=DescriptionStyle(description_width=u'initial'), _dom_classes=('space', 'blue')), Label(value=u'word_lexicon: None', description=u'None', style=DescriptionStyle(description_width=u'initial'), _dom_classes=('space', 'blue')), Label(value=u'conv_size: 50', description=u'50', style=DescriptionStyle(description_width=u'initial'), _dom_classes=('space', 'blue')), Label(value=u'word_window_size: 5', description=u'5', style=DescriptionStyle(description_width=u'initial'), _dom_classes=('space', 'blue')), Label(value=u'adagrad: True', description=u'True', style=DescriptionStyle(description_width=u'initial'), _dom_classes=('space', 'blue')), Label(value=u'normalization: minmax', description=u'minmax', style=DescriptionStyle(description_width=u'initial'), _dom_classes=('space', 'blue'))), layout=Layout(align_self=u'center', display=u'inline-flex', flex_flow=u'column', width=u'95%')), Label(value=u'Caracteristicas Multiplas', description=u'Caracteristicas Multiplas', layout=Layout(align_self=u'flex-start', left=u'2%', width=u'91%'), _dom_classes=('top-line',)), VBox(children=(Label(value=u'Word_embedding', description=u'Word_embedding', layout=Layout(align_self=u'center', width=u'95%'), _dom_classes=('space',)), HBox(children=(Checkbox(value=False, description=u'/data/ner/ptwiki_cetenfolha_cetempublico.trunk.voc5.100wv.ctw5.sample1e_1', indent=False, style=DescriptionStyle(description_width=u'word_embedding'), _dom_classes=('space',)), Checkbox(value=False, description=u'/dpgs-data/ner/data/ptwiki_cetenfolha_cetempublico.trunk.voc5.100wv.ctw5.sample1e_1', indent=False, style=DescriptionStyle(description_width=u'word_embedding'), _dom_classes=('space',)), Checkbox(value=False, description=u'/home/eraldo/lia/src/lia-pln-datasets-models/ner/data/ptwiki_cetenfolha_cetempublico.trunk.voc5.100wv.ctw5.sample1e_1', indent=False, style=DescriptionStyle(description_width=u'word_embedding'), _dom_classes=('space',))), layout=Layout(display=u'inline-flex', flex_flow=u'column', width=u'